# Bidirectional LSTM (Long Short Term Memory)
### Sentiment Analysis - The Simpsons

In [14]:
import kagglehub

from os import path
import chardet
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import plotly
import plotly.express as px

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

import string
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nrclex import NRCLex


In [3]:
path = kagglehub.dataset_download("prashant111/the-simpsons-dataset")
file_name = "simpsons_script_lines.csv"
file_path = path + "\\" + file_name
with open(file_path, 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [4]:
df = pd.read_csv(file_path, low_memory=False)
df.head(3)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,true,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,true,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,true,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22


In [5]:
df = df.dropna()
df.head()

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,true,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,true,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,true,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,true,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,true,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33


In [6]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\mia.jensen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\mia.jensen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\mia.jensen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\mia.jensen\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\mia.jensen\AppDa

True

In [7]:
description_list=[]
for description in df['normalized_text']:
    description=re.sub("[^a-zA-Z]", " ", description)
    description=description.lower()
    description=nltk.word_tokenize(description)
    description=[word for word in description if not word in set(stopwords.words("english"))]
    lemma=nltk.WordNetLemmatizer()
    description=[lemma.lemmatize(word) for word in description] 
    description=" ".join(description)
    description_list.append(description)

df["normalized_text_new"]=description_list
df.head(5)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,normalized_text_new
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,true,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31,actually little sometimes disease magazine new...
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,true,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3,wheres mr bergstrom
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,true,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22,dont know although id sure like talk didnt tou...
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,true,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5,life worth living
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,true,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33,poll open end recess case decided put thought ...


In [8]:
# val_homer=[]
# val_bart=[]
# val_marge=[]
# val_lisa=[]

# for i, row in df.iterrows():
#     val = row['normalized_text_new']
#     if row['character_id'] == "2":
#         val_homer.append(val)
#     elif row['character_id'] == "8":
#         val_bart.append(val)
#     elif row['character_id'] == "1":
#         val_marge.append(val)
#     elif row['character_id'] == "9":
#         val_lisa.append(val)   

# stop = stopwords.words('english')
# pat = r'\b(?:{})\b'.format('|'.join(stop))

# def text_cleaning(val_list):
#     df1 = pd.DataFrame(val_list,columns =['normalized_text_new']).dropna()
#     df1["normalized_text_new"] = df1["normalized_text_new"].str.replace(pat, '')
#     df1["normalized_text_new"] = df1["normalized_text_new"].str.replace(r'\s+', ' ')
#     return df1

# bart = text_cleaning(val_bart)
# homer = text_cleaning(val_homer)
# marge = text_cleaning(val_marge)
# lisa = text_cleaning(val_lisa)

In [9]:
text_object = NRCLex(' '.join(df['normalized_text_new']))

In [52]:
df_test = df.copy()
df_test["nrclex_object"] = df_test['normalized_text_new'].apply(lambda val: NRCLex(val))
df_test["affect_frequencies"] = df_test["nrclex_object"].apply(lambda val: val.affect_frequencies)
df_test["top_emotions"] = df_test["nrclex_object"].apply(lambda val: val.top_emotions)
df_test

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,normalized_text_new,nrclex_object,affect_frequencies,top_emotions
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,true,464,3.0,Miss Hoover,Springfield Elementary School,"No, actually, it was a little of both. Sometim...",no actually it was a little of both sometimes ...,31,actually little sometimes disease magazine new...,<nrclex.NRCLex object at 0x000001CCA20728D0>,"{'fear': 0.16666666666666666, 'anger': 0.16666...","[(fear, 0.16666666666666666), (anger, 0.166666..."
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,true,9,3.0,Lisa Simpson,Springfield Elementary School,Where's Mr. Bergstrom?,wheres mr bergstrom,3,wheres mr bergstrom,<nrclex.NRCLex object at 0x000001CCA20736D0>,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...","[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t..."
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,true,464,3.0,Miss Hoover,Springfield Elementary School,I don't know. Although I'd sure like to talk t...,i dont know although id sure like to talk to h...,22,dont know although id sure like talk didnt tou...,<nrclex.NRCLex object at 0x000001CCD1AF51D0>,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...","[(positive, 0.3333333333333333)]"
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,true,9,3.0,Lisa Simpson,Springfield Elementary School,That life is worth living.,that life is worth living,5,life worth living,<nrclex.NRCLex object at 0x000001CCD1A67B10>,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...","[(positive, 1.0)]"
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,true,40,3.0,Edna Krabappel-Flanders,Springfield Elementary School,The polls will be open from now until the end ...,the polls will be open from now until the end ...,33,poll open end recess case decided put thought ...,<nrclex.NRCLex object at 0x000001CCCF4F5510>,"{'fear': 0.14285714285714285, 'anger': 0.0, 'a...","[(trust, 0.2857142857142857)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158266,9544,32,204,Miss Hoover: (OFF LISA'S REACTION) I'm back.,831000,true,464,3.0,Miss Hoover,Springfield Elementary School,I'm back.,im back,2,im back,<nrclex.NRCLex object at 0x000001CCFB938710>,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...","[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t..."
158267,9545,32,205,"Miss Hoover: You see, class, my Lyme disease t...",839000,true,464,3.0,Miss Hoover,Springfield Elementary School,"You see, class, my Lyme disease turned out to ...",you see class my lyme disease turned out to be,10,see class lyme disease turned,<nrclex.NRCLex object at 0x000001CCFB938B90>,"{'fear': 0.2, 'anger': 0.2, 'anticip': 0.0, 't...","[(fear, 0.2), (anger, 0.2), (negative, 0.2), (..."
158268,9546,32,206,Miss Hoover: Psy-cho-so-ma-tic.,842000,true,464,3.0,Miss Hoover,Springfield Elementary School,Psy-cho-so-ma-tic.,psy-cho-so-ma-tic,1,psy cho tic,<nrclex.NRCLex object at 0x000001CCFB938BD0>,"{'fear': 0.0, 'anger': 0.0, 'anticip': 0.0, 't...","[(fear, 0.0), (anger, 0.0), (anticip, 0.0), (t..."
158269,9547,32,207,Ralph Wiggum: Does that mean you were crazy?,844000,true,119,3.0,Ralph Wiggum,Springfield Elementary School,Does that mean you were crazy?,does that mean you were crazy,6,mean crazy,<nrclex.NRCLex object at 0x000001CCFB9397D0>,"{'fear': 0.25, 'anger': 0.25, 'anticip': 0.0, ...","[(fear, 0.25), (anger, 0.25), (negative, 0.25)..."


In [48]:
df_test[df_test["NRCLex_top"]!="fear"]

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,normalized_text_new,NRCLex,NRCLex_freq,NRCLex_top


In [10]:
text_object.affect_frequencies

{'fear': 0.08591706051143556,
 'anger': 0.0667936112752131,
 'anticip': 0.0,
 'trust': 0.11878375812304835,
 'surprise': 0.05517079500379779,
 'positive': 0.18751582412017892,
 'negative': 0.1470350873491434,
 'sadness': 0.07324985230821167,
 'disgust': 0.059986602244915185,
 'joy': 0.09940448561060006,
 'anticipation': 0.10614292345345598}

In [11]:
text_object.top_emotions

[('positive', 0.18751582412017892)]

In [12]:
sentiment_scores = pd.DataFrame(list(text_object.raw_emotion_scores.items())) 

In [13]:
sentiment_scores = sentiment_scores.rename(columns={0: "Sentiment", 1: "Count"})
sentiment_scores

,Sentiment,Count
0,anger,25326
1,disgust,22745
2,fear,32577
3,negative,55751
4,sadness,27774
5,trust,45039
6,positive,71100
7,anticipation,40246
8,joy,37691
9,surprise,20919


In [19]:
sentiment = sentiment_scores['Sentiment'].to_list()

In [20]:
sentiment_words = pd.DataFrame(list(text_object.affect_dict.items()), columns = ['words','sentiments'])
sentiment_words

,words,sentiments
0,disease,"[anger, disgust, fear, negative, sadness]"
1,show,[trust]
2,talk,[positive]
3,lesson,"[anticipation, positive, trust]"
4,plan,[anticipation]
...,...,...
4773,authoritative,"[positive, trust]"
4774,misnomer,[negative]
4775,sullen,"[anger, negative, sadness]"
4776,concentric,[positive]


In [21]:
for y in sentiment:
    sentiment_words[y] = 9
sentiment_words

,words,sentiments,anger,disgust,fear,negative,sadness,trust,positive,anticipation,joy,surprise
0,disease,"[anger, disgust, fear, negative, sadness]",9,9,9,9,9,9,9,9,9,9
1,show,[trust],9,9,9,9,9,9,9,9,9,9
2,talk,[positive],9,9,9,9,9,9,9,9,9,9
3,lesson,"[anticipation, positive, trust]",9,9,9,9,9,9,9,9,9,9
4,plan,[anticipation],9,9,9,9,9,9,9,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...
4773,authoritative,"[positive, trust]",9,9,9,9,9,9,9,9,9,9
4774,misnomer,[negative],9,9,9,9,9,9,9,9,9,9
4775,sullen,"[anger, negative, sadness]",9,9,9,9,9,9,9,9,9,9
4776,concentric,[positive],9,9,9,9,9,9,9,9,9,9


In [25]:
sentiment

['anger',
 'disgust',
 'fear',
 'negative',
 'sadness',
 'trust',
 'positive',
 'anticipation',
 'joy',
 'surprise']

In [28]:
a=0
for sents in sentiment_words['sentiments']: 
    for sent in sentiment:
        sentiment_words.loc[a, sent] = int(sent in sents)
    a=a+1
    break

sentiment_words

,words,sentiments,anger,disgust,fear,negative,sadness,trust,positive,anticipation,joy,surprise
0,disease,"[anger, disgust, fear, negative, sadness]",1,1,1,1,1,0,0,0,0,0
1,show,[trust],0,0,0,0,0,1,0,0,0,0
2,talk,[positive],0,0,0,0,0,0,1,0,0,0
3,lesson,"[anticipation, positive, trust]",0,0,0,0,0,1,1,1,0,0
4,plan,[anticipation],0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4773,authoritative,"[positive, trust]",0,0,0,0,0,1,1,0,0,0
4774,misnomer,[negative],0,0,0,1,0,0,0,0,0,0
4775,sullen,"[anger, negative, sadness]",1,0,0,1,1,0,0,0,0,0
4776,concentric,[positive],0,0,0,0,0,0,1,0,0,0


In [34]:
for sent in sentiment:
    word_list = sentiment_words[sentiment_words[sent]== 1].words.head(10)
    print(f"Sentiment:{sent}")
    print(f"{word_list.values}\n")

Sentiment:anger
['disease' 'backbone' 'vote' 'demand' 'fraud' 'pretending' 'ill' 'lie'
 'hate' 'hurt']

Sentiment:disgust
['disease' 'ill' 'lie' 'hate' 'baboon' 'feeling' 'gag' 'boy' 'painful'
 'toxic']

Sentiment:fear
['disease' 'case' 'spike' 'sweat' 'god' 'ill' 'problem' 'abandon' 'hate'
 'hurt']

Sentiment:negative
['disease' 'case' 'touched' 'vote' 'demand' 'leave' 'fraud' 'pretending'
 'ill' 'lie']

Sentiment:sadness
['disease' 'case' 'vote' 'leave' 'ill' 'lie' 'problem' 'abandon' 'cry'
 'hate']

Sentiment:trust
['show' 'lesson' 'teach' 'poll' 'statement' 'victory' 'backbone' 'vote'
 'president' 'teacher']

Sentiment:positive
['talk' 'lesson' 'teach' 'worth' 'statement' 'victory' 'job' 'traditional'
 'backbone' 'vote']

Sentiment:anticipation
['lesson' 'plan' 'thought' 'victory' 'vote' 'long' 'track' 'delight'
 'coming' 'tomorrow']

Sentiment:joy
['teach' 'victory' 'vote' 'delight' 'god' 'true' 'glad' 'feeling'
 'confidence' 'happy']

Sentiment:surprise
['teach' 'vote' 'leave' 'g